# Task 1:  Poyp Segmentation  (Testing and Interpretation)

## Test Results Generation

In [ ]:
import glob
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from matplotlib import image
from numpy import savez_compressed
from torchvision import transforms
import copy


import torch
from gan import Generator, Discriminator
from torch.utils.data import DataLoader, Dataset
from torch.autograd import Variable
from PIL import Image
import torch.nn.functional as F
os.environ['KMP_DUPLICATE_LIB_OK']='True'

import random


In [ ]:
ngf= 64
ndf= 64
lrG=0.002 #0.0002
lrD=0.002
lamb=100
beta1=0.5
beta2=0.999

In [ ]:
G = Generator(3, ngf, 3)
D = Discriminator(6, ndf, 1)
G.normal_weight_init(mean=0.0, std=0.02)
D.normal_weight_init(mean=0.0, std=0.02)

print(D)

print(G)

In [ ]:
def save_mask_batch(gen_masks_batch, 
                     msk_fn_batch,
                     gen_msk_dir_name,
                    original_size):
    
   
    for i in range(len(gen_masks_batch)):
        
        msk_fn=msk_fn_batch
        
        gen_mask=gen_masks_batch[i].detach().numpy()#.permute(1,2,0).detach().numpy().astype(np.uint8)

       
        save_genmsk_fn = gen_msk_dir_name + msk_fn
#         print(save_genmsk_fn)
        gen_mask_ = (((gen_mask -gen_mask.min()) * 255) / (gen_mask.max() - gen_mask.min())).transpose(1, 2, 0).astype(np.uint8)
        msk_gen = Image.fromarray(gen_mask_)
        msk_gen_original_size= msk_gen.resize((original_size[0],original_size[1]), Image.ANTIALIAS)
        msk_gen_original_size.save(save_genmsk_fn)
        #plt.show()
        plt.close()

        
        
        
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.5, 0.5, 0.5],
        std =[0.5, 0.5, 0.5],
    ),
])

In [ ]:
# loading test data

test_data_dir = "./data/task1/test_data/"
test_results_dir = "./data/task1/test_data/masks/"

test_img_files = glob.glob(test_data_dir+"images/*.jpg")
#msk_files=glob.glob(data_dir+"masks/*.jpg")


In [ ]:
#loading trained model, we only need the generator

PATH="saved_models/Seg_GAN_task1.pth"
model_ckp = torch.load(PATH)
G.load_state_dict(model_ckp['g_state_dict'],strict=True)
#D.load_state_dict(checkpoint['d_state_dict'],strict=True)

In [ ]:
# generate  masks

target_w=512
target_h=512

for  i,f in enumerate(test_img_files):
    img = Image.open(f)
# 1. resize images to the inputs size
    image_size=list(img.size) # we need to keep the original image size
    img_resized = img.resize((target_w,target_h), Image.ANTIALIAS)
    img_tr=transform(img_resized.copy()).numpy()
    test_image= torch.tensor(img_tr)
    test_image = torch.unsqueeze(test_image, 0)
# 2. generate  512 * 512 mask
    
    gen_msak_sample,_= G(test_image)
    
    
    im_file_name=os.path.basename(f)
# save the generated mask, we also pass the original size to get the original shape    
    save_mask_batch(gen_msak_sample,im_file_name,test_results_dir,image_size)

   
    


## Interpreting Model Test Set

We reference the paper for more details about layer relevance propagation, the approach we used to interperate the models with their implementation guides:

* Montavon, Grégoire, Alexander Binder, Sebastian Lapuschkin, Wojciech Samek, and Klaus-Robert Müller. "Layer-wise relevance propagation: an overview." Explainable AI: interpreting, explaining and visualizing deep learning (2019): 193-209.

In [ ]:
def transform_back(im):
    im_ = (((im -im.min()) * 255) / (im.max() - im.min())).transpose(1, 2, 0).astype(np.uint8)
    return(im_)

def new_layer(layer, g):
    #copy a layer and pass its parameters through the function g."
    layer = copy.deepcopy(layer)
    try: layer.weight = torch.nn.Parameter(g(layer.weight))
    except AttributeError: pass
    
    try: layer.bias = torch.nn.Parameter(g(layer.bias))
    except AttributeError: pass
    return layer

In [ ]:
target_w=512
target_h=512


for  i,f in enumerate(test_img_files):
    img = Image.open(f)
    image_size=list(img.size)
    img_resized = img.resize((target_w,target_h), Image.ANTIALIAS)
    img_tr=transform(img_resized.copy()).numpy()
    test_image= torch.tensor(img_tr)
    test_image = torch.unsqueeze(test_image, 0)
    
    gen_msak_sample,_= G(test_image)
    
    
    im_file_name=os.path.basename(f)
    

    layers=list(G._modules) # get generator trained layers (parameters)

    
    n_layers = len(layers)
    
# set a place holder for the activations to calculates next
    activations = [test_image] + [None] * n_layers # list of activations


    # for each layer  in the conv part of the generator
    for layer in range(9):
        
        #get the activations
        activation = G._modules[layers[layer]].forward(activations[layer])
        
        #pass it for the next layer
        activations[layer+1] = activation
    
    # for each layer  in the deconv part of the generator
    for layer in range(9,len(layers)):
        
        #convatenate the corresponding conv layer based on the skip connections
        cat_in=torch.cat([activations[layer], activations[16-layer]], 1) 
        activation = G._modules[layers[layer]].forward(cat_in)
        activations[layer+1] = activation

            
    # calculate the output as we use tanh in the model
    output_activation = torch.nn.Tanh()(activations[-1]).detach().cpu().numpy()#[0,:,:,:]

    # convert the RGB mask output to grey scale
    outpu_activation=np.asarray(Image.fromarray(output_activation[0,:,:,:].transpose(1,2,0).astype('uint8'),
                                               'RGB').convert('L'))
    
    # set predicted mask pixels to 1
    output_activation[output_activation>=0]= 1
    # set predicted normal pixels to 0
    output_activation[output_activation!=1]= 0
    
    # save the output activation to the last element of the activations placeholder
    activations[-1]=torch.FloatTensor(output_activation) 

    # Backpropagate relevvance scores
    relevances = [None] * n_layers + [activations[-1]]
    
    # go over the layers in reverse order
    
    # define the rho function based on the model of haveing <17 layers (refer to the above paper)
    rho = lambda p: p + 0.25*p.clamp(min=0); incr = lambda z: z+1e-9

    for layer in range(0, n_layers)[::-1]:
        # inreverse order we start by the deconv section
        if layer>=9:

            activations[layer] = activations[layer].data.requires_grad_(True)

            cat_in2=torch.cat([activations[layer], activations[16-layer]], 1)
            z = incr(new_layer(G._modules[layers[layer]],rho).forward(cat_in2))
            
            # devide relevance / z element wise
            s = (relevances[layer+1]/z).data    
            
            # Calculate the gradient and multiply it by the activation,
            # we should retain the graph as we will pass through it multiple times
            (z * s).sum().backward(retain_graph=True); 
            c = activations[layer].grad       
            relevances[layer] = (activations[layer]*c).data
        # we do the same for the conv section
        else:
            activations[layer] = activations[layer].data.requires_grad_(True)

            
            cat_in2=torch.cat([activations[layer], activations[layer]], 1)
            z = incr(new_layer(G._modules[layers[layer]],rho).forward(cat_in2[:,:int(cat_in2.shape[1]/2),:,:]))

            s = (relevances[layer+1]/z).data                                     
            (z * s).sum().backward(retain_graph=True); 
            c = activations[layer].grad       
            relevances[layer] = (activations[layer]*c).data                          

    
    
    # plotting and saving relevance scores
    img_to_plot=transform_back(test_image[0,:].detach().numpy())
    gen_msak_sample_to_plot=transform_back(gen_msak_sample[0,:].detach().numpy())
    fig, axs= plt.subplots(1,3,figsize=(30,20))

    axs[0].imshow(img_to_plot)
    axs[0].axis('off')
    axs[0].set_title("Image", size=24)

    image_rlp=relevances[0][0,:,:,:].permute(1,2,0).detach().numpy()
    image_rlp = np.interp(image_rlp, (image_rlp.min(),
                                                    image_rlp.max()), 
                                                    (0, 1))
    axs[1].imshow(image_rlp[:,:,0], cmap="seismic")

    axs[1].axis('off')

    axs[1].set_title("Input Pixels Relevance ", size=24)

    axs[2].imshow(gen_msak_sample_to_plot)
    axs[2].axis('off')
    axs[2].set_title("Predicted mask", size=24)
    
    
    plt.show()

    
    
    fig.savefig('rlp_results_task1/input_pix/'+im_file_name, dpi=200)

    
    selected_layers=[0,4,6,12,13,15]

    fig, axs= plt.subplots(1,len(selected_layers)+1,figsize=(30,20))

    axs[0].imshow(img_to_plot)
    axs[0].axis('off')
    axs[0].set_title("Image", size=24)
    for l,layer in enumerate(selected_layers):

        image_rlp=relevances[layer][0,:,:,:].permute(1,2,0).detach().numpy()
        image_rlp = np.interp(image_rlp, (image_rlp.min(),
                                                        image_rlp.max()), 
                                                        (0, 1))
        axs[l+1].imshow(image_rlp[:,:,0], cmap="seismic")


        axs[l+1].axis('off')


        axs[l+1].set_title("layer "+str(layer), size=24)

    axs[l+1].imshow(gen_msak_sample_to_plot)
    axs[l+1].axis('off')
    axs[l+1].set_title("predicted mask", size=24)


    plt.show()
    
    fig.savefig('rlp_results_task1/all_layers/'+im_file_name, dpi=200)
